# XOR문제 학습하기

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import activations
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers

In [2]:
class XOR(keras.Model):
    """XOR Network"""
    def __init__(self):
        super(XOR, self).__init__()
        # 층을 구성
        self.dense_layers = keras.Sequential()
        self.dense_layers.add(layers.Dense(units=300, activation=activations.relu))
        self.dense_layers.add(layers.Dense(units=300, activation=activations.relu))
        self.dense_layers.add(layers.Dense(units=2, activation=activations.linear))
    
    def call(self, x):
        # forward propagation 수행
        y = self.dense_layers(x)
        return y

In [3]:
targets = tf.convert_to_tensor([0, 1, 1, 0], dtype=tf.int32)
targets = tf.one_hot(targets, depth=2)

In [7]:
tf.random.set_seed(777)

n_step = 10001  # 총 학습 스텝

# Data 세트 만들기
inputs = tf.convert_to_tensor([[0, 0], [1, 0], [0, 1], [1, 1]], dtype=tf.float32)
targets = tf.convert_to_tensor([1, 0, 0, 1], dtype=tf.int32)
targets = tf.one_hot(targets, depth=2)

# 모델 생성
model = XOR()

# 손실함수
loss_function = losses.BinaryCrossentropy(from_logits=True)

# 옵티마이져
optimizer = optimizers.SGD(learning_rate=0.001)

best_loss = 999
# n_step 동안 학습을 진행한다.
for step in range(n_step):
    # -- 훈련단계 --
    train_loss = 0
    # 정답을 작성해주세요.
    with tf.GradientTape() as tape:
        # (1) 순방향전파
        output = model(inputs)
        # (2) 손실값 계산
        loss_value = loss_function(targets,output)
        # (3) 역방향전파(Back Propagation)
    grads = tape.gradient(loss_value, model.variables)
    # ) 옵티마이저로 매개변수 업데이트
    optimizer.apply_gradients(zip(grads,model.variables))
    
    # 훈련단계 손실값 기록(모든 데이터에 손실값의 평균을 합친다.)
    train_loss += loss_value.numpy()
    if train_loss < best_loss:
        best_loss = train_loss
        model.save_weights('./XOR_model.h5')
    if step % 5000 == 0:
        print(f"[{step+1}] Loss: {train_loss:.4f}")
        print(model(inputs).numpy())
        print(tf.argmax(model(inputs), axis=1).numpy())

[1] Loss: 0.6940
[[ 3.3525444e-05 -3.7491449e-05]
 [ 3.1072076e-02 -4.7213096e-02]
 [-2.4196273e-02 -1.9563939e-02]
 [ 3.0168166e-02 -5.4448713e-02]]
[0 0 1 0]
[5001] Loss: 0.6057
[[-0.08901623  0.09503485]
 [ 0.28645587 -0.2691916 ]
 [ 0.20570996 -0.23869753]
 [-0.14452897  0.14860308]]
[1 0 0 1]
[10001] Loss: 0.4755
[[-0.24235433  0.25027993]
 [ 0.6379852  -0.64354515]
 [ 0.5816383  -0.6124146 ]
 [-0.52095777  0.5415581 ]]
[1 0 0 1]


In [8]:
test_data = inputs = tf.convert_to_tensor([[1, 0], [0, 0], [1, 1], [0, 1]], dtype=tf.float32)

pred =  model(test_data)

print(f"Test Data Result : {tf.argmax(pred,axis = 1).numpy()}")

Test Data Result : [0 1 1 0]
